In [19]:
from tensorflow.keras.layers import Input, LSTM, GRU, SimpleRNN, Dense, GlobalMaxPool1D,Reshape,Dropout,Lambda
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras import layers,models,Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LeakyReLU
import tensorflow.initializers as initializers 

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_val_score
# from lightgbm import LGBMClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold
# from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [20]:
files_path = 'CleanFiles/'
files_format = '.csv'
files_postfix = 'Extract'
# file_names = ['AgiaParaskevi','Aristotelous','Elefsina',
#               'Lykovrisi','Marousi','NeaSmirni','Peristeri',
#               'Pireus','Thrakomakedones']
file_names = [
              'Marousi',
              'Aristotelous',
              'NeaSmirni',
              'AgiaParaskevi',
              'Pireus',
              'Peristeri'
             ]

file_names_nogrid = [
                'Thrakomakedones',
                'Elefsina'
            ]

grid = {'Marousi': ['Lykovrisi','Peristeri','AgiaParaskevi']
        ,'Aristotelous': ['Marousi','Peristeri','NeaSmirni']
        ,'AgiaParaskevi': ['Marousi','NeaSmirni','Koropi']
        ,'NeaSmirni': ['Aristotelous','Pireus','AgiaParaskevi']
        ,'Pireus': ['Elefsina','Peristeri','NeaSmirni']
        ,'Peristeri':['Thrakomakedones','Marousi','Pireus']
       }


grid2 = {'Marousi': ['Lykovrisi','Aristotelous']
        ,'Aristotelous': ['Marousi','Peristeri']
        ,'AgiaParaskevi': ['Marousi','Koropi']
        ,'NeaSmirni': ['Aristotelous','Pireus']
        ,'Pireus': ['Elefsina','NeaSmirni']
        ,'Peristeri':['Thrakomakedones','Marousi']
       }

In [21]:
def pm10_index(val):
    if val <= 25.0:
        return 0
    elif 26.0 <= val <= 50.0:
        return 1
    elif 51.0 <= val <= 90.0:
        return 2
    elif 91.0 <= val <= 180.0:
        return 3
    else:
        return 4

In [22]:
def get_time(tmp):
    return tmp[11:13]

In [23]:
def get_season(tmp):
    if tmp == 'Spring':
        return '1'
    elif tmp == 'Spring/Summer':
        return '2'
    elif tmp == 'Summer':
        return '3'
    elif tmp == 'Summer/Autumn':
        return '4'
    elif tmp == 'Autumn':
        return '5'
    elif tmp == 'Autumn/Winter':
        return '6'
    elif tmp == 'Winter/Spring':
        return '7'
    else:
        return '8'

In [24]:
def get_winddir(tmp):
    if tmp == 'N':
        return '1'
    elif tmp == 'NNE':
        return '2'
    elif tmp == 'NE':
        return '3'
    elif tmp == 'ENE':
        return '4'
    elif tmp == 'E':
        return '5'
    elif tmp == 'ESE':
        return '6'
    elif tmp == 'SE':
        return '7'
    elif tmp == 'SSE':
        return '8'
    elif tmp == 'S':
        return '9'
    elif tmp == 'SSW':
        return '10'
    elif tmp == 'SW':
        return '11'
    elif tmp == 'WSW':
        return '12'
    elif tmp == 'W':
        return '13'
    elif tmp == 'WNW':
        return '14'
    elif tmp == 'NW':
        return '15'
    else:
        return '16'

In [25]:
#neasmirni = pd.read_csv('CleanFiles/NeaSmirniExtract.csv', sep=',')
#neasmirni = neasmirni[['date_time','station_id','season','real_temp','real_windspd','real_winddir','real_humidity','forecast_tempC','forecast_windSpeed','forecast_windDirection','forecast_humidity','pm10']]
#neasmirni['pm10'] = neasmirni['pm10'].map(lambda a: pm10_index(a))
#temp = pd.read_csv('CleanFiles/PeristeriExtract.csv', sep=',')
#agiaparaskevi = pd.read_csv('CleanFiles/AgiaParaskeviExtract.csv', sep=',')

In [26]:
# T = 72
# D = 1
# X = []
# Y = []
# for t in range(len(neasmirni) - T):
#     x = neasmirni[t:t+T]
#     X.append(x)
#     y = neasmirni[t:t+T]
#     Y.append(y)

In [27]:
def create_xy(series, series2, window_size, prediction_horizon, shuffle = False):
    x = []
    y = []
    for i in range(0, len(series)):
        if len(series[(i + window_size):(i + window_size + prediction_horizon)]) < prediction_horizon:
            break
        x.append(np.array(series[i:(i + window_size)]))
        y.append(np.array(series2[(i + window_size):(i + window_size + prediction_horizon)]))
    x = np.array(x)
    y = np.array(y)
    return x,y

In [28]:
parameters = {
    'n_estimators': 2000,
    'max_depth': 4,
    'num_leaves': 2**4,
    'learning_rate': 0.1,
    'boosting_type': 'dart'
}

In [29]:
def smape(actual, predicted):
    if not all([isinstance(actual, np.ndarray), 
                isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual),
        np.array(predicted)
  
    return round(
        np.mean(
            np.abs(predicted - actual) / 
            ((np.abs(predicted) + np.abs(actual))/2)
        )*100, 2
    )

In [30]:
def gru(units,drop,features, window, horizon):
    encoder_inputs = layers.Input(shape=(window,features))
    encoder = GRU(units, dropout=drop,return_state=True)
    _,encoder_states = encoder(encoder_inputs)
    decoder=layers.RepeatVector(horizon)(encoder_states)
    out = layers.TimeDistributed(Dense(1))(decoder)
    model = models.Model(encoder_inputs, out)
    model.compile(loss='mse', optimizer=RMSprop())
    return model

In [31]:
def seq2seq(window, horizon, units,bn,drop,feautures,channel=1):
    encoder_inputs = layers.Input(shape=(window,feautures))

    encoder = GRU(units, dropout=drop,return_state=True)
    _,encoder_states = encoder(encoder_inputs)
    if bn:
        encoder_states=layers.BatchNormalization()(encoder_states)
    decoder=layers.RepeatVector(horizon)(encoder_states)
    decoder_gru = GRU(units, dropout=drop, return_sequences=True, return_state=False)
    decoder = decoder_gru(decoder, initial_state=encoder_states)
    
    out = layers.TimeDistributed(Dense(channel,use_bias=True))(decoder)
    model = models.Model(encoder_inputs, out)
    model.compile(loss='mse', optimizer=RMSprop(),metrics=['mean_squared_error'])
    return model

In [32]:
def gatedDNN(units,features, horizon):
#     model = Sequential()
#     model.add(layers.Input(shape=(72,features)))
#     model.add(layers.Dense(2, activation='relu'))
#     model.compile(loss='mean_squared_error', optimizer='adam')
#     return model

    model = Sequential([
    Lambda(lambda x: x[:, -1:, :]),
    Dense(units, activation='relu',use_bias=True),
    Dense(horizon*1),
    Reshape([horizon, 1])
    ])
    model.add(LeakyReLU(alpha=0.01))
    model.compile(loss='mse', optimizer=RMSprop(),metrics=['mean_squared_error'])
    return model

In [36]:
def lstm(features, window, horizon):
    model = Sequential([
    LSTM(10, input_shape=(window, features), return_sequences=True),
    Dropout(0.5),
    LSTM(10, return_sequences=True),
    Dense(horizon),
    Reshape([horizon, 1])
    ])
    
    model.compile(optimizer=RMSprop(), loss='mse', metrics=['mean_squared_error'])
    return model

In [16]:
def lightgbm():
    model = lgb.LGBMRegressor(first_metric_only = True)
    return model

In [59]:
def loadToModel(files, formating, pathFiles, postfx, params):
    T = 72
    D = 1
    X = []
    Y = []
    for fileName in files:
        window = 72 
        horizon = 48
        accuracy = 0
        #df = pd.read_csv(pathFiles + fileName + postfx + formating, sep=',')
        df = pd.read_csv(pathFiles + fileName + formating, sep=',')
        df['date_time'] = df['date_time'].astype("|S")
        df['season'] = df['season'].astype("|S")
        df['date_time'] = df['date_time'].map(lambda a: get_time(a))
        df['date_time'] = df['date_time'].astype(int)
        df['season'] = df['season'].map(lambda a: get_season(a))
        df['season'] = df['season'].astype(np.float64)
        df['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df['station_id'] = df['station_id'].astype(int)
        df2 = df[['pm10']]

        scaler = MinMaxScaler(feature_range=(-3, 3))
        names = ['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']
        df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']] = scaler.fit_transform(df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']])
        
        
        scaler1 = MinMaxScaler(feature_range=(-3, 3))
        df2 = scaler1.fit_transform(df2.values)
        

        X,y = create_xy(df[['station_id','date_time','forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']],df2,window, horizon)
        accuracy = 0.0
        
        print(fileName + ' begins prediction')
#         model = seq2seq(72,48,50,True,0.5,7,1)
#         model = gru(5,0.5,7, 72, 48)
#         model = lstm(5, window, horizon)
#         model = lightgbm()
        model = gatedDNN(100,7, horizon)

        kf = KFold(n_splits=5)

#         print(X.shape)
#         print(y.shape)
        
        epochs = 2
        batch_size = 32

        predicts = []
        tests = []
#         for trainI,testI in kf.split(X):
#             model.fit(X[trainI],y[trainI], epochs=epochs, batch_size=batch_size)
#             predictions = model.predict(X[testI])
#             predictions = scaler.inverse_transform(predictions.reshape(-1,1)).reshape(predictions.shape)
#             predicts.append(predictions)
#             tests.append(y[testI])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        model.fit(X_train,y_train, epochs=epochs, batch_size=batch_size)
        predictions = model.predict(X_test)
        predictions = scaler1.inverse_transform(predictions.reshape(-1,1)).reshape(predictions.shape)
        y_test = scaler1.inverse_transform(y_test.reshape(-1,1)).reshape(y_test.shape)

    return predictions, y_test

In [167]:
def loadToModelGrid2(files, formating, pathFiles, postfx, params):
    T = 72
    D = 1
    X = []
    Y = []
    for fileName in files:
        window = 72 
        horizon = 48
        accuracy = 0
        characteristics = 10
        df = pd.read_csv(pathFiles + fileName + formating, sep=',')
        df1 = pd.read_csv(pathFiles + grid2[fileName][0] + formating, sep=',')
        df2 = pd.read_csv(pathFiles + grid2[fileName][1] + formating, sep=',')
        df['date_time'] = df['date_time'].astype("|S")
        df['season'] = df['season'].astype("|S")
        df['date_time'] = df['date_time'].map(lambda a: get_time(a))
        df['date_time'] = df['date_time'].astype(int)
        df['season'] = df['season'].map(lambda a: get_season(a))
        df['season'] = df['season'].astype(np.float64)
        df['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df1['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df2['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df['station_id'] = df['station_id'].astype(int)
        df['windir1'] = df1['forecast_windDirection']
        df['pm101'] = df1['pm10']
        df['windir2'] = df2['forecast_windDirection']
        df['pm102'] = df2['pm10']
        
        df2 = df[['pm10']]

        scaler = MinMaxScaler(feature_range=(-3, 3))
        names = ['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10','windir1','pm101','windir2','pm102']
        df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10','windir1','pm101','windir2','pm102']] = scaler.fit_transform(df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10','windir1','pm101','windir2','pm102']])
        
        
        scaler1 = MinMaxScaler(feature_range=(-3, 3))
        df2 = scaler1.fit_transform(df2.values)
        

        X,y = create_xy(df[['station_id','date_time','forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10','windir1','pm101','windir2','pm102']],df2,window, horizon)
        accuracy = 0.0
        
        print(fileName + ' begins prediction')
#         model = seq2seq(72,48,50,True,0.5,15,1)
#         model = gru(5,0.5,15, 72, 48)
#         model = lstm(15, window, horizon)
#         model = lightgbm()
        model = gatedDNN(100,characteristics, horizon)

        kf = KFold(n_splits=5)

#         print(X.shape)
#         print(y.shape)
        
        epochs = 2
        batch_size = 32

        predicts = []
        tests = []
#         for trainI,testI in kf.split(X):
#             model.fit(X[trainI],y[trainI], epochs=epochs, batch_size=batch_size)
#             predictions = model.predict(X[testI])
#             predictions = scaler.inverse_transform(predictions.reshape(-1,1)).reshape(predictions.shape)
#             predicts.append(predictions)
#             tests.append(y[testI])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        model.fit(X_train,y_train, epochs=epochs, batch_size=batch_size)
        predictions = model.predict(X_test)
        predictions = scaler1.inverse_transform(predictions.reshape(-1,1)).reshape(predictions.shape)
        y_test = scaler1.inverse_transform(y_test.reshape(-1,1)).reshape(y_test.shape)

    return predictions, y_test

In [60]:
def loadToModelGrid3(files, formating, pathFiles, postfx, params):
    T = 72
    D = 1
    X = []
    Y = []
    for fileName in files:
        window = 72 
        horizon = 48
        accuracy = 0
        characteristics = 12
        #df = pd.read_csv(pathFiles + fileName + postfx + formating, sep=',')
        df = pd.read_csv(pathFiles + fileName + formating, sep=',')
        df1 = pd.read_csv(pathFiles + grid[fileName][0] + formating, sep=',')
        df2 = pd.read_csv(pathFiles + grid[fileName][1] + formating, sep=',')
        df3 = pd.read_csv(pathFiles + grid[fileName][2] + formating, sep=',')
#         df4 = pd.read_csv(pathFiles + grid[fileName][3] + formating, sep=',')
        df['date_time'] = df['date_time'].astype("|S")
        df['season'] = df['season'].astype("|S")
        df['date_time'] = df['date_time'].map(lambda a: get_time(a))
        df['date_time'] = df['date_time'].astype(int)
        df['season'] = df['season'].map(lambda a: get_season(a))
        df['season'] = df['season'].astype(np.float64)
        df['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df1['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df2['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df3['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df['station_id'] = df['station_id'].astype(int)
        df['windir1'] = df1['forecast_windDirection']
        df['pm101'] = df1['pm10']
        df['windir2'] = df2['forecast_windDirection']
        df['pm102'] = df2['pm10']
        df['windir3'] = df3['forecast_windDirection']
        df['pm103'] = df3['pm10']
        
        df2 = df[['pm10']]

        scaler = MinMaxScaler(feature_range=(-3, 3))
        names = ['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10','windir1','pm101','windir2','pm102','windir3','pm103']
        df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10','windir1','pm101','windir2','pm102','windir3','pm103']] = scaler.fit_transform(df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10','windir1','pm101','windir2','pm102','windir3','pm103']])
        
        
        scaler1 = MinMaxScaler(feature_range=(-3, 3))
        df2 = scaler1.fit_transform(df2.values)
        

        X,y = create_xy(df[['station_id','date_time','forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10','windir1','pm101','windir2','pm102','windir3','pm103']],df2,window, horizon)
        accuracy = 0.0
        
        print(fileName + ' begins prediction')
#         model = seq2seq(72,48,50,True,0.5,15,1)
#         model = gru(5,0.5,15, 72, 48)
#         model = lstm(15, window, horizon)
#         model = lightgbm()
        model = gatedDNN(100,characteristics, horizon)

        kf = KFold(n_splits=5)

#         print(X.shape)
#         print(y.shape)
        
        epochs = 2
        batch_size = 32

        predicts = []
        tests = []
#         for trainI,testI in kf.split(X):
#             model.fit(X[trainI],y[trainI], epochs=epochs, batch_size=batch_size)
#             predictions = model.predict(X[testI])
#             predictions = scaler.inverse_transform(predictions.reshape(-1,1)).reshape(predictions.shape)
#             predicts.append(predictions)
#             tests.append(y[testI])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        model.fit(X_train,y_train, epochs=epochs, batch_size=batch_size)
        predictions = model.predict(X_test)
        predictions = scaler1.inverse_transform(predictions.reshape(-1,1)).reshape(predictions.shape)
        y_test = scaler1.inverse_transform(y_test.reshape(-1,1)).reshape(y_test.shape)

    return predictions, y_test

In [61]:
predicts, test = loadToModelGrid3(file_names, files_format, files_path, files_postfix, parameters)
#predicts, test = loadToModel(file_names, files_format, files_path, files_postfix, parameters)

Marousi begins prediction
Train on 29298 samples
Epoch 1/2
29298/29298 [==============================] - 2s 67us/sample - loss: 1.6972 - mean_squared_error: 1.6973
Epoch 2/2
29298/29298 [==============================] - 1s 45us/sample - loss: 0.0335 - mean_squared_error: 0.0335
Aristotelous begins prediction
Train on 29298 samples
Epoch 1/2
29298/29298 [==============================] - 3s 111us/sample - loss: 1.7708 - mean_squared_error: 1.7708
Epoch 2/2
29298/29298 [==============================] - 2s 54us/sample - loss: 0.0381 - mean_squared_error: 0.0381
NeaSmirni begins prediction
Train on 29298 samples
Epoch 1/2
29298/29298 [==============================] - 2s 69us/sample - loss: 1.8491 - mean_squared_error: 1.8491
Epoch 2/2
29298/29298 [==============================] - 1s 36us/sample - loss: 0.0345 - mean_squared_error: 0.0345
AgiaParaskevi begins prediction
Train on 29298 samples
Epoch 1/2
29298/29298 [==============================] - 2s 68us/sample - loss: 1.9829 - mean_

In [62]:
predicts[10000]

array([[34.762573],
       [36.228054],
       [35.547977],
       [36.42726 ],
       [34.328747],
       [34.26845 ],
       [33.89342 ],
       [34.64176 ],
       [33.878635],
       [33.53098 ],
       [32.732063],
       [32.658928],
       [31.328953],
       [30.997265],
       [31.58164 ],
       [31.80794 ],
       [31.178661],
       [30.175596],
       [27.941797],
       [28.386301],
       [28.12144 ],
       [31.219963],
       [32.514305],
       [34.27798 ],
       [34.535446],
       [35.10698 ],
       [35.275246],
       [35.25389 ],
       [34.306217],
       [32.023914],
       [32.044117],
       [31.166998],
       [32.79593 ],
       [34.728664],
       [33.40701 ],
       [32.820213],
       [32.334312],
       [31.23287 ],
       [30.772177],
       [29.698046],
       [28.147987],
       [26.665459],
       [26.1975  ],
       [25.736967],
       [27.034115],
       [28.304302],
       [29.595142],
       [31.024353]], dtype=float32)

In [63]:
test[10000]

array([[41.],
       [36.],
       [35.],
       [49.],
       [39.],
       [53.],
       [39.],
       [52.],
       [40.],
       [41.],
       [36.],
       [36.],
       [38.],
       [38.],
       [36.],
       [39.],
       [53.],
       [57.],
       [45.],
       [43.],
       [38.],
       [45.],
       [43.],
       [40.],
       [34.],
       [33.],
       [34.],
       [39.],
       [32.],
       [38.],
       [39.],
       [44.],
       [41.],
       [42.],
       [46.],
       [43.],
       [49.],
       [37.],
       [41.],
       [69.],
       [44.],
       [49.],
       [41.],
       [43.],
       [36.],
       [30.],
       [23.],
       [25.]])

In [64]:
print(np.amin(predicts))
print(np.amin(test))
print(np.amax(predicts))
print(np.amax(test))

5.7313476
0.0
244.58075
802.0


In [65]:
predicts, test = loadToModel(file_names_nogrid, files_format, files_path, files_postfix, parameters)

Thrakomakedones begins prediction
Train on 29298 samples
Epoch 1/2
29298/29298 [==============================] - 2s 66us/sample - loss: 2.1921 - mean_squared_error: 2.1921
Epoch 2/2
29298/29298 [==============================] - 2s 72us/sample - loss: 0.0317 - mean_squared_error: 0.0317
Elefsina begins prediction
Train on 29298 samples
Epoch 1/2
29298/29298 [==============================] - 2s 60us/sample - loss: 1.9745 - mean_squared_error: 1.9745
Epoch 2/2
29298/29298 [==============================] - 2s 79us/sample - loss: 0.0261 - mean_squared_error: 0.0261


In [66]:
predicts[10000]

array([[33.76148 ],
       [33.288223],
       [32.977478],
       [32.255966],
       [32.176544],
       [30.928398],
       [30.904318],
       [30.741457],
       [30.745644],
       [31.980553],
       [32.65741 ],
       [32.68418 ],
       [33.23324 ],
       [32.902626],
       [32.460056],
       [31.036873],
       [30.931824],
       [30.225178],
       [30.595177],
       [30.673643],
       [30.331287],
       [31.70411 ],
       [32.260696],
       [33.463562],
       [33.855137],
       [33.71019 ],
       [33.489815],
       [32.820896],
       [30.640839],
       [30.055386],
       [29.56221 ],
       [29.390678],
       [29.23214 ],
       [29.884344],
       [30.70153 ],
       [31.447777],
       [30.789646],
       [31.180763],
       [31.940735],
       [31.041794],
       [30.29609 ],
       [29.839172],
       [29.84719 ],
       [29.666092],
       [29.58534 ],
       [31.788338],
       [31.711826],
       [32.198807]], dtype=float32)

In [67]:
test[10000]

array([[37.],
       [39.],
       [36.],
       [35.],
       [32.],
       [32.],
       [33.],
       [31.],
       [32.],
       [29.],
       [30.],
       [50.],
       [40.],
       [34.],
       [51.],
       [33.],
       [41.],
       [43.],
       [40.],
       [32.],
       [37.],
       [34.],
       [34.],
       [33.],
       [30.],
       [38.],
       [35.],
       [45.],
       [37.],
       [34.],
       [32.],
       [31.],
       [30.],
       [31.],
       [29.],
       [37.],
       [43.],
       [44.],
       [58.],
       [50.],
       [48.],
       [35.],
       [32.],
       [33.],
       [30.],
       [25.],
       [31.],
       [25.]])

In [68]:
print(np.amin(predicts))
print(np.amin(test))
print(np.amax(predicts))
print(np.amax(test))

6.804551
0.0
164.82494
684.0


In [71]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [ ]:
def return_axes(df):
    x = np.arange(df.shape[0])[:, None, None]
    y = np.arange(df.shape[1])[None, :, None]
    z = np.arange(df.shape[2])[None, None, :]
    x, y, z = np.broadcast_arrays(x, y, z)
    return x,y,z

In [ ]:
# x, y, z = return_axes(predicts)
# fig = plt.figure()
# ax = fig.gca(projection='3d')
# ax.scatter(x.ravel(),
#            y.ravel(),
#            z.ravel())

In [ ]:
# x, y, z = return_axes(test)
# fig = plt.figure()
# ax = fig.gca(projection='3d')
# ax.scatter(x.ravel(),
#            y.ravel(),
#            z.ravel())